<a href="https://colab.research.google.com/github/dohyun93/hongong_mldl/blob/main/5_2_%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 검증 세트
# 테스트세트를 사용하지 않으면 과대적합인지 과소적합인지 알기가 어렵다.
# 테스트 세트를 사용하지 않고 측정하는 간단한 방법은 훈련세트를 또 나누는것이다. 이 데이터를 검증 세트라고 한다.

# 즉 전체 데이터가 100이라고 했을 때, 훈련세트를 60, 검증세트를 20, 테스트세트를 20으로 구성한다.
# (데이터가 많다면 적은비율로 해도 데이터 대표성을 띄기 때문에 괜찮다.)

import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class'].to_numpy()

print("전체 데이터 shape:", data.shape)
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
#train_input, train_target 을 훈련세트와 검증 세트로 만든다.
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print("훈련, 검증, 테스트세트 shape:", sub_input.shape, val_input.shape, test_input.shape)
# sub_input: 훈련세트
# val_input: 검증세트

전체 데이터 shape: (6497, 3)
훈련, 검증, 테스트세트 shape: (4157, 3) (1040, 3) (1300, 3)


In [3]:
 from sklearn.tree import DecisionTreeClassifier
 dt = DecisionTreeClassifier(random_state=42)
 dt.fit(sub_input, sub_target)
 print("1. 훈련세트 스코어: ", dt.score(sub_input, sub_target))
 print("2. 검증세트 스코어: ", dt.score(val_input, val_target))

1. 훈련세트 스코어:  0.9971133028626413
2. 검증세트 스코어:  0.864423076923077


In [4]:
# 검증 세트를 만드느라 훈련세트가 줄었다.
# 보통 많은 데이터를 훈련에 사용할 수록 좋은 모델이 만들어지는데 그렇다고 검증세트를 너무 조금 떼어내면 검증점수가 들쭉날쭉하고 불안정할 것이다.
# 이럴 때 '교차 검증' (cross-validation) 을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.

# K-fold 교차 검증

# 교차 검증

교차 검증은 훈련세트를 훈련세트와 검증세트로 나눈 뒤 이를 번갈아 교차해가면서 훈련시키는 방법이다.

훈련 세트를 세 부분으로 나누어 훈련/검증 세트로 나누고 이를 번갈아 가며 훈련시킨 뒤 검증점수를 평균내어 학습된 성능을 확인한다.

In [6]:
# 보통 5-fold, 10-fold 교차 검증을 많이 사용한다.
# 여기서는 예시로 3-Fold 교차검증을 살펴본다.

# 사이킷런에는 cross_validate()라는 교차 검증 함수가 있다. 
# 평가할 모델 객체와 훈련/검증 세트로 나누기 전의 훈련세트를 통째로 전달한다.

from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01265717, 0.02270961, 0.01634765, 0.01693034, 0.01257277]), 'score_time': array([0.01507068, 0.00324726, 0.00327516, 0.00351644, 0.00342679]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [7]:
# cross_validate 함수는 기본적으로 5-fold 교차검증을 수행한다.
# 'cv' 매개변수로 이를 조절할 수 있다.

# fit_time: 훈련세트로 모델을 훈련하는데 걸린 시간
# score_time: 검증세트로 모델을 평가하는데 걸린 시간
# test_score: 교차검증에서 k-fold의 각 경우에 대한 검증세트의 점수

# 따라서 test_score 점수의 평균이 곧 이 k-fold 교차검증의 점수이다.
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


> 유의할 점은 교차검증 함수 cross_validate() 는 훈련세트를 섞어 폴드를 나누지 않는다.

> 앞서 train_test_split 으로 훈련세트를 섞었기 때문에 여기서는 따로 섞을 필요가 없지만, 만약 앞서 그런 단계가 없었을 경우 '분할기(splitter)' 를 지정해야 한다.

> 사이킷런에서 분할기는 교차검증에서 폴드를 어떻게 나눌지 결정해준다. cross_validate() 는 기본적으로 회귀 모델일 경우 ```KFold``` 분할기를 사용하고, 분류 모델일 경우 ```StratifiedKFold``` 를 사용한다. (stratify: 층을 이루다)

In [9]:
# 분류 모델이기 때문에 위 셀과 점수가 동일하다는 걸 확인할 수 있다.
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# 만약 10-fold 교차 검증을 하려면 어떻게 할까?
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print("10fold cross-validate scores: ", np.mean(scores['test_score']))

10fold cross-validate scores:  0.8574181117533719


# 하이퍼파라미터 튜닝

머신러닝 모델이 학습하는 파라미터를 ```모델 파라미터``` 라고 부른다고 했다.

하지만 머신러닝 개발자가 직접 조절해야 하는 파라미터를 ```하이퍼 파라미터``` 라고 부른다고 했는데, 이 값이 다수일 경우 한 파라미터를 고정하고 다른 파라미터를 조절하는게 아니라, ```동시에 하이퍼파라미터를 바꿔가며 최적의 값을 찾아야 한다.```

```GridSearchCV``` 는 교차검증과 하이퍼파라미터 최적 값 찾기를 동시에 수행해준다. 별도로 cross_validate 를 호출할 필요가 없다.

In [ ]:
# 1. GridSearchCV import
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}


# 1. GridSearchCV import
from sklearn.model_selection import GridSearch